In [23]:
import os 
import sys 
import warnings
import numpy as np 
import scipy.ndimage as ndimage
from UtilsFunc import dirlistfiles, gdal_edit_ndv

from glob import glob 
from pygeotools.lib import iolib
from osgeo import gdal
from scipy import stats

warnings.filterwarnings('ignore')



def get_itemfromlist(fs, K):
    f = [i for i in fs if K in i]
    print(f[0])
    return f[0]

In [24]:
tile_path = '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/'
fs = dirlistfiles(tile_path,'tif', True)

subdirectiries included
26


In [25]:
tdxdem_path = get_itemfromlist(fs, 'TDX_DEM.tif')
tdxcom_path = get_itemfromlist(fs, 'TDX_COM.tif')
copwbm_path = get_itemfromlist(fs, 'COP_WBM.tif')




/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/N10_E105_TDX_DEM.tif
/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/N10_E105_TDX_COM.tif
/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/N10_E105_COP_WBM.tif


In [27]:
def tdx_noise_removal(tdxdem_path, tdxcom_path, copwbm_path):
    ndv_vals =-32767.
    ndv_msks = 0.0 

    print('Applying ndv')
    gdal_edit_ndv(copwbm_path, ndv_msks)
    gdal_edit_ndv(tdxcom_path, ndv_msks)
    gdal_edit_ndv(tdxdem_path, ndv_vals)

    print('Applying mask for DEM')
    dem_fn = tdxdem_path #glob.glob(os.path.join(tiledir, 'DEM/*DEM.tif'))[0]
    print(dem_fn)
    dem_ds = iolib.fn_getds(dem_fn)
    dem = iolib.ds_getma(dem_ds)
    print(dem.count())
    #Get original mask, True where masked
    mask = np.ma.getmaskarray(dem) 

    print('Applying mask for WBM')
    wbm = iolib.fn_getma(copwbm_path)
    wbm_invalid = (1,2,3)
    mask = np.logical_or(mask, np.isin(wbm.data, wbm_invalid))

    print('Applying mask for COM')
    com_fn = tdxcom_path#glob.glob(os.path.join(tiledir, 'AUXFILES/*COM.tif'))[0]
    com = iolib.fn_getma(com_fn)
    com_valid = (8,9,10)
    #4 is only one obs
    #com_invalid = (0,1,4)
    com_invalid = (0,1)
    mask = np.logical_or(mask, np.isin(com.data, com_invalid))

    print('Applying mask filter for TDX')
    dem_masked = np.ma.array(dem, mask=mask)
    print(dem_masked.count())

    tif_masked = tdxdem_path.replace('.tif', '_masked.tif')
    iolib.writeGTiff(dem_masked, tif_masked, dem_ds)

    print('Applying morphological mask filter for TDX')
    n_iter = 1
    mask = ndimage.morphology.binary_dilation(mask, iterations=n_iter)
    #To keep valid edges, do subsequent erosion 
    mask = ndimage.morphology.binary_erosion(mask, iterations=n_iter)
    #(dilation of inverted mask, to avoid maasking outer edge)
    #mask = ~(ndimage.morphology.binary_dilation(~mask, iterations=n_iter))
    tif_erode = tdxdem_path.replace('.tif', '_erode.tif')
    iolib.writeGTiff(dem_masked, tif_erode, dem_ds)
    gdal_edit_ndv(tif_erode, -9999.)
    gdal_edit_ndv(tif_masked, -9999.)

    return tif_erode, tif_masked


    


In [28]:
tif_erode, tif_masked = tdx_noise_removal(tdxdem_path, tdxcom_path, copwbm_path)

Applying ndv
Applying mask for DEM
/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/N10_E105_TDX_DEM.tif
80230518
Applying mask for WBM
Applying mask for COM
Applying mask filter for TDX
76442881


Warning 1: PROJ: proj_create_from_database: Open of /home/ljp238/miniconda3/envs/automl/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /home/ljp238/miniconda3/envs/automl/share/proj failed


Applying morphological mask filter for TDX


ERROR 1: PROJ: proj_create_from_name: Open of /home/ljp238/miniconda3/envs/automl/share/proj failed


In [29]:
fpath = '/media/ljp238/6tb/Joseph/DATASETS/Mekong/delta_s_m2070.tif'

In [31]:
from UtilsTiling import *

In [32]:
get_geotiff_nodatavalue(fpath)


-3.4028230607370965e+38